# Модель Transformer-2

Взять тот же датасет и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2. Сделать генерацию заголовков для статьи

In [ ]:
# Загрузка данных

!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [ ]:
# Функция чтения данных

import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [ ]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [ ]:
train_records[0].keys()

dict_keys(['url', 'text', 'title', 'summary', 'date'])

In [ ]:
print(min([record["date"] for record in train_records]), end=' - ')
print(max([record["date"] for record in train_records]))
print(min([record["date"] for record in val_records]), end=' - ')
print(max([record["date"] for record in val_records]))
print(min([record["date"] for record in test_records]), end=' - ')
print(max([record["date"] for record in test_records]))

2010-06-01 10:35:49 - 2019-05-31 23:56:26
2019-06-01 08:30:00 - 2019-09-30 23:11:23
2019-10-01 08:23:02 - 2020-03-23 22:16:23


In [ ]:
test_records[0]

{'date': '2020-02-14 16:39:11',
 'summary': 'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.',
 'text': 'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторы

In [ ]:
import pandas as pd

dataset = pd.DataFrame(test_records)
dataset.head(3)

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5770 entries, 0 to 5769
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      5770 non-null   object
 1   text     5770 non-null   object
 2   title    5770 non-null   object
 3   summary  5770 non-null   object
 4   date     5770 non-null   object
dtypes: object(5)
memory usage: 225.5+ KB


In [ ]:
dataset = dataset.drop(['url', 'date'], axis=1)

In [ ]:
# Сделаем маленький тестовый датасет 

dataset = dataset[:20]

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     20 non-null     object
 1   title    20 non-null     object
 2   summary  20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


## Summarization

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.0 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
     |████████████████████████████████| 101 kB 8.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 8.4 MB/s 


### Модель MBARTRuSumGazeta

In [ ]:
# Загрузка модели 

# from transformers import MBartTokenizer, MBartForConditionalGeneration

# model_name = "IlyaGusev/mbart_ru_sum_gazeta"
# tokenizer = MBartTokenizer.from_pretrained(model_name)
# model = MBartForConditionalGeneration.from_pretrained(model_name)

from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

In [ ]:
# Пример работы модели

article_text = dataset['text'][1]

input_ids = tokenizer(
    [article_text],
    max_length=600,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)

print('Predicted summary: ', summary)
print('Real summary: ', dataset['summary'][1])

Predicted summary:  В Кремле прошло праздничное шоу «Танцуют все!» на телеканале «Россия». Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов и эвенков.
Real summary:  25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.


In [ ]:
# Генерация краткого содержания для всего датасета

data = dataset['text']

summaries = []

for row in data:
    input_ids = tokenizer(
        [row],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    summaries.append(summary)

In [ ]:
dataset['pred_summary'] = summaries

In [ ]:
dataset.sample(3)

,text,title,summary,pred_summary
16,В 2019 году на Кубу прибыло рекордное число ту...,Дешевая экзотика: почему россияне уезжают на Кубу,Россияне стали чаще отдыхать на Кубе. За прошл...,"В 2019 году на Кубу прибыло 178 тыс. россиян, ..."
0,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",NASA огласило названия четырех космических мис...
18,"Американскому круизному лайнеру Westerdam, на ...",«Нет больных? Откуда знаете?» Лайнер из США ни...,"Круизный лайнер Westerdam, принадлежащий амери...","Американскому круизному лайнеру Westerdam, на ..."


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 7.2 MB/s 
     |████████████████████████████████| 212 kB 37.8 MB/s 
     |████████████████████████████████| 140 kB 51.1 MB/s 
     |████████████████████████████████| 1.1 MB 39.0 MB/s 
     |████████████████████████████████| 127 kB 41.8 MB/s 
     |████████████████████████████████| 271 kB 43.7 MB/s 
     |████████████████████████████████| 144 kB 39.6 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Загрузка библиотек для оценки качества 

from datasets import load_metric

rouge_score = load_metric("rouge")

In [ ]:
predictions = dataset['pred_summary']
references = dataset['summary']

In [ ]:
# Rouge

sum_scores = rouge_score.compute(
    predictions=[predictions], references=[references]
)
sum_scores

{'rouge1': AggregateScore(low=Score(precision=0.38461538461538464, recall=0.75, fmeasure=0.5084745762711865), mid=Score(precision=0.38461538461538464, recall=0.75, fmeasure=0.5084745762711865), high=Score(precision=0.38461538461538464, recall=0.75, fmeasure=0.5084745762711865)),
 'rouge2': AggregateScore(low=Score(precision=0.21052631578947367, recall=0.42105263157894735, fmeasure=0.2807017543859649), mid=Score(precision=0.21052631578947367, recall=0.42105263157894735, fmeasure=0.2807017543859649), high=Score(precision=0.21052631578947367, recall=0.42105263157894735, fmeasure=0.2807017543859649)),
 'rougeL': AggregateScore(low=Score(precision=0.358974358974359, recall=0.7, fmeasure=0.4745762711864407), mid=Score(precision=0.358974358974359, recall=0.7, fmeasure=0.4745762711864407), high=Score(precision=0.358974358974359, recall=0.7, fmeasure=0.4745762711864407)),
 'rougeLsum': AggregateScore(low=Score(precision=0.358974358974359, recall=0.7, fmeasure=0.4745762711864407), mid=Score(prec